# Variational Autoencoders (VAEs)

## Helpful Identities

* $I$ is the identity matrix
* $\log(A \cdot B) = \log A + \log B$
* A **neural network** $f(\cdot; \theta)$ is a function (defined by a composition of simple functions), parameterized by weights $\theta$.
* **Marginalization:** given $p(x, z)$, the marginal is $p(x) = \mathbb{E}_{p(z)} [ p(x | z) ]$
* **KL-divergence** a measure of "distance" between distributions: $\text{KL}[q(z) || p(z)] = \mathbb{E}_{q(z)} \left[ \log \frac{q(z)}{p(z)} \right] \geq 0$
* **Jensen's Inequality:** $\log \mathbb{E}_{p(z)} [f(z)] \geq \mathbb{E}_{p(z)} [\log f(z)]$

## Introduction 

**Motivation:**
* Want to be able to generate realistic-looking data (e.g. airport security)
* Want to be able to evaluate the probability of a data-point

**Question:** How can we design model + inference to accomplish this task?

**Today:** We are going to combine several topics from the course with some tools from deep learning to define a model + inference capable of expressing very complicated distributions, like a distribution of images. 

The topics we rely on today are:
1. Latent variable models
2. Maximum Likelihood (MLE)
3. Expectation-Maximization (EM)
4. Bayesian Inference (i.e. inferring posterior)
5. Gradient descent
6. Sampling 
7. Monte Carlo (MC) estimation of expectations


## Overview

1. Introduce the model
2. Demo
3. Inference: why EM doesn't work
4. Variational Inference
5. Ticks for scalability

## The Model

**Model:** Variational Autoencoder / Non-linear Factor Analysis
\begin{align}
z &\sim p(z) = \mathcal{N}(0, I) \\
x | z &\sim p(x | z; \theta) = \mathcal{N}(f(z; \theta), \sigma^2 \cdot I)
\end{align} in which $x_1, \dots, x_N \in \mathbb{R}^D$ is observed, $z_1, \dots, z_N \in \mathbb{R}^L$ is latent, $f(\cdot; \theta)$ is some function (e.g. neural network). 

When $f(z; \theta)$ is a linear function, this model is known as **factor analysis**. 

**Picture for intuition:** TODO.

**Inference Goal:** maximize the likelihood of the data under the model (MLE)
\begin{align}
\max\limits_\theta \frac{1}{N} \sum\limits_{n=1}^N \log p(x_n; \theta)
\end{align}

## Demo

**Links:** 
* [Demo](http://taylordenouden.com/VAE-Latent-Space-Explorer/)
* [Cool visualization](https://arxiv.org/pdf/1704.03477.pdf)

**What do we see in the demo?**
* The demo consists of a model trained to generate hand-drawn digits.
* It visualizes the latent space of the model -- that is, the projection of each image $x$ onto the 2D $z$-space.

**Questions:**
1. What does each color in the latent space represent?
2. What happens when you move from a region with one color to another?
3. What do you notice about how $x$ transforms as you move your cursor (and how would it differ from a naive interpolation between images)? 


## Inference

**Can we use the naive MLE?** MLE for this model is intractable:
\begin{align}
\max\limits_\theta \frac{1}{N} \sum\limits_{n=1}^N \log p(x_n; \theta)
&= \max\limits_\theta \frac{1}{N} \sum\limits_{n=1}^N \log \underbrace{\mathbb{E}_{p(z_n)} [ p(x_n | z_n; \theta) ]}_{\text{intractable}}
\end{align}

Why is it intractable? (draw picture).
* Case 1: high variance MC-estimate
* Case 2: low variance computationally expensive MC-estimate

### Can we apply EM?

Let's derive it:

\begin{align}
\log p(x_n; \theta) 
&= \log \mathbb{E}_{p(z_n)} [ p(x_n | z_n; \theta) ] \\
&= \log \int\limits_{z_n} p(x_n | z_n; \theta) p(z_n) d z_n \\
&= \log \int\limits_{z_n} q(z_n) \cdot \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} d z_n \\
&= \log \mathbb{E}_{q(z_n)} \left[ \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} \right] \quad \text{Importance Sampling: how to select } q(z_n)? \\
&\geq \mathbb{E}_{q(z_n)} \left[ \log \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} \right] \quad \text{Soln: optimize lower bound jointly w.r.t } \theta, q(z_n)
\end{align} Plugging this back into our MLE goal, we get:
\begin{align}
\underbrace{\frac{1}{N} \sum\limits_{n=1}^N \log p(x_n; \theta)}_{\text{log marginal/incomplete likelihood}} \geq \underbrace{\frac{1}{N} \sum\limits_{n=1}^N \mathbb{E}_{q(z_n)} \left[ \log \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} \right]}_{\text{ELBO}(\theta, q(z_1), \dots, q(z_N))}
\end{align}

In EM, this is the objective we optimize via coordinate ascent. Before applying coordinate ascent, let's understand the **tightness** of this bound.
\begin{align}
\frac{1}{N} \sum\limits_{n=1}^N \log p(x_n; \theta) 
&= \underbrace{\frac{1}{N} \sum\limits_{n=1}^N  \mathbb{E}_{q(z_n)} \left[ \log \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} \right]}_{\text{ELBO}(\theta, q(z_1), \dots, q(z_N))} + \underbrace{\frac{1}{N} \sum\limits_{n=1}^N \text{KL} \left[ q(z_n) || p(z_n | x_n; \theta) \right]}_{\geq 0 \quad \text{"gap"}}
\end{align} The above equation shows that maximizing the ELBO is **equivalent** to maximizing the likelihood when the "gap" is 0 -- i.e. when we can set $q(z_n)$ equal to the posterior exactly.

We apply **coordinate ascent** as follows:
* **E-step:**
\begin{align}
q^\text{new}(z_1) &= \mathrm{argmax}_{q(z_1)} \quad \text{ELBO}(\theta^\text{old}, q(z_1), \dots, q^\text{old}(z_N)) \\
&\vdots \\
q^\text{new}(z_N) &= \mathrm{argmax}_{q(z_N)} \quad \text{ELBO}(\theta^\text{old}, q^\text{old}(z_1), \dots, q(z_N))
\end{align}
* **M-step:**
\begin{align}
\theta^\text{new} = \mathrm{argmax}_{\theta} \quad &\text{ELBO}(\theta, q^\text{old}(z_1), \dots, q^\text{old}(z_N)) 
\end{align}
* **Repeat!**

What's the solution to the E-step? $q^\text{new}(z_n) = p(z_n | x_n; \theta^\text{old})$. But this requires a **closed-form** solution, which we do not have in general. How can we get around this?
* Sampling (e.g. MH)? (take a long time, scales poorly with $N$)

### Variational Inference

Since **coordinate ascent** relies on conjugacy, what other optimization techniques can we use to maximize the ELBO? Can we use gradient descent?

We can! But we'll need to make a few changes to our inference (in 3 parts).

**Part 1: Select form for $q(z_n)$.** Previously, we set $q^\text{new}(z_n) = p(z_n | x_n; \theta^\text{old})$ using an analytic formula. However, when such a formula does not exist, what can we do? Let's approximate the posterior.

That is, we'll set $q(z_n)$ to come from some family of distributions that's easy to work with (i.e. that we can sample from and for which we can evaluate the PDF). This distribution will not capture the posterior perfectly, but will hopefully capture it well enough. We'll call this family of distributions a **variational family**. One common choice is a diagonal Gaussian:
\begin{align}
q(z_n) = \mathcal{N}(\mu_n, \sigma^2_n)
\end{align}

Now, when we maximize the ELBO, we do so by taking gradients with respect to the parameters of $q(z_n)$, as opposed to relative to $q(z_n)$:
\begin{align}
\nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \text{ELBO}(\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N) 
\end{align}

**Part 2: Reparameterization Trick.**
\begin{align}
\nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \text{ELBO}(\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N)
&= \nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \frac{1}{N} \sum\limits_{n=1}^N \mathbb{E}_{q(z_n)} \left[ \log \frac{p(x_n | z_n; \theta) p(z_n)}{q(z_n)} \right] \\
&\approx \nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \frac{1}{N} \sum\limits_{n=1}^N \frac{1}{S} \sum\limits_{s=1}^S \left[ \log \frac{p(x_n | z_n^{(s)}; \theta) p(z_n^{(s)})}{q(z_n^{(s)})} \right]
\end{align} where $z_n^{(s)} \sim q(z_n) = \mathcal{N}(\mu_n, \sigma^2_n)$. 

But how can we take a gradient with respect to a **sample**, $z_n^{(s)}$? We can use the following identity (know as the **reparameterization trick**):
\begin{align}
z_n^{(s)} &= r(\eta_n^{(s)}, \sigma_n, \mu_n) \\
&= \eta_n^{(s)} \cdot \sigma_n + \mu_n, \quad \eta_n^{(s)} \sim \mathcal{N}(0, I)
\end{align} Plugging this in, we get:
\begin{align}
\nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \text{ELBO}(\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N)
&= \nabla_{\theta, \mu_1, \sigma_1, \dots, \mu_N, \sigma_N} \frac{1}{N} \sum\limits_{n=1}^N \mathbb{E}_{p(\eta)} \left[ \log \frac{p(x_n | r(\eta_n^{(s)}, \sigma_n, \mu_n); \theta) p(r(\eta_n^{(s)}, \sigma_n, \mu_n))}{q(r(\eta_n^{(s)}, \sigma_n, \mu_n))} \right] \\
\end{align} in which we do not need to take derivative with respect to the samples.

**Part 3: Amortize Inference.** The above gradients are over a very large number of parameters (specifically $N$), which does not scale well. Can we make this more efficient?

Yes! By **amortizing**: we set $q(z_n) = \mathcal{N}(\mu(x_n; \phi), \sigma^2(x_n; \phi))$, where $\mu(\cdot; \phi), \sigma^2(\cdot; \phi)$ are neural networks parameterized by $\phi$. Now, we only need to compute gradients with respect to $\theta$ and $\phi$.
\begin{align}
\nabla_{\theta, \phi} \text{ELBO}(\theta, \phi)
\end{align}

## Putting it all together 

<img src="https://miro.medium.com/max/982/1*6MxYa1CLA4Kuy0i65Wc4sw.png" />
